# Updated 

In [47]:
from PIL import Image
import cv2
import pytesseract
import numpy as np
from fuzzywuzzy import fuzz
import fitz
import os
import glob
import time
import img2pdf
import shutil
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt

start = time.time()

pdf_file = 'Batch 1-test.pdf'

create_path = '/home/vimalkumar/Downloads/new_project (copy)/output'


in_path = os.path.join('/home/vimalkumar/Downloads/new_project (copy)/input_images',pdf_file)
out_path = os.path.join(create_path,str(os.path.splitext(pdf_file)[0]))
out_path_images = os.path.join(out_path,str('Images'))
out_path_files = os.path.join(out_path,str('Files'))

os.makedirs(out_path)
os.makedirs(out_path_images)
os.makedirs(out_path_files)

doc = fitz.open(in_path)

for i in range(len(doc)):
    doc = fitz.open(in_path)
    page = doc.load_page(i)  # number of page
    pix = page.get_pixmap(matrix=fitz.Matrix(150/72,150/72))# for 150dpi
    file_name = str(i+1000 + 1)       
    pix.pil_save(os.path.join(out_path_images,file_name+'.jpeg'))
    
count = []
empty = []
for img in sorted(os.listdir(os.path.join(out_path_images))):
    print(img)
    im = cv2.imread(os.path.join(out_path_images,img),0)
#     _, blackAndWhite = cv2.threshold(im, 127, 255, cv2.THRESH_BINARY_INV)

#     nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
#     sizes = stats[1:, -1] #get CC_STAT_AREA component
#     img2 = np.zeros((labels.shape), np.uint8)

#     for i in range(0, nlabels - 1):
#         if sizes[i] >= 50:   #filter small dotted regions
#             img2[labels == i + 1] = 255

#     res = cv2.bitwise_not(img2)
    #im = cv2.imread('/home/vimalkumar/Downloads/new_project/output_image/1002.jpeg',0)
    image_data = pytesseract.image_to_string(im,config=r'--oem 3 --psm 6')
    #print(image_data)

    out = " ".join(image_data.splitlines()).lower()
    #print(out)
    
    out_replace = out.replace('  ','')
    #print(len(out_replace))
    
    image_data_2 = pytesseract.image_to_string(im)
    out_1 = " ".join(image_data_2.splitlines()).lower()
    #print(out_1)
    
    out_1_replace = out_1.replace(' ','')
    #print(len(out_1_replace))
    
    
    if len(out_replace) < 15 or len(out_1_replace) < 15:
        empty.append(os.path.splitext(img)[0])
        #else:
            #continue
   
    else:
        fuzz_ratio = fuzz.partial_ratio(out,'new document to follow')
        fuzz_ratio_1 = fuzz.partial_ratio(out,'new file to follow')
       
        #print(fuzz_ratio)

        if fuzz_ratio >= 80 or fuzz_ratio_1 >= 80:
            count.append(os.path.splitext(img)[0])
            print('new document')
count.append(int(os.path.splitext(img)[0])+1)

c = []
for b in count:
    c.append(int(b))
    
diff = list(np.diff(c))
diff.append(0)
#diff

if len(c) == 1 and len(diff) == 1:
    cc = str(c)
    cc = cc.replace('[','')
    cc = cc.replace(']','')

    check_cc = int(cc[1:]) - 1
    
else:
    check_cc = 0.01

dic = dict(zip(c,diff))

key = list(dic.keys())[0]
value = list(dic.values())[0]

val_diff = key - 1001



if val_diff ==  check_cc:
    #dicc = {1001:val_diff}
    #dic = {**dicc, **dic}
    key_list= []
    value_list=[]
    for x,y in dic.items():
        key_list.append(x)
        value_list.append(y)
    key_list[0] = 1001
    value_list[0] = val_diff + value
    
    dicc = dict(zip(key_list,value_list))
    dic = {**dicc, **dic}
    dic_length = len(dic) - 1
    
    
elif val_diff != 0:
    key_list= []
    value_list=[]
    for x,y in dic.items():
        key_list.append(x)
        value_list.append(y)
    key_list[0] = 1001
    value_list[0] = val_diff + value
    
    dicc = dict(zip(key_list,value_list))
    dic = {**dicc, **dic}
    dic_length = len(dic) - 2
    
else:
    dic = dic
    dic_length = len(dic) - 1
    
    
#print(dic)

empty_pdf = 'empty_pdf'
empty_folder_path = os.path.join(out_path_files,empty_pdf)
os.makedirs(empty_folder_path)

cnt = 1
for j in range(dic_length):
    file = list(dic.keys())[j]
    #print(file)
    file_from = str(file)
    file_to = str(list(dic.values())[j] + file - 1)
    
    #folder_name = str(file)+'-'+str(list(dic.values())[j] + file - 1)
    folder_name = file_from[1:] + '-' + file_to[1:]    
        
    folder_path = os.path.join(out_path_files,folder_name)
    os.makedirs(folder_path)    
    
    
    for k in range(list(dic.values())[j]):
        #print(k)
        empty_check = file+k
        str_empty_check = str(empty_check)       
        
        if str(empty_check) in empty:
            #pass
            image = cv2.imread(os.path.join(out_path_images,str(empty_check)+'.jpeg'))
            #plt.imshow(image)
            imm = Image.fromarray(image)
            #imm.save("your_file.jpeg")
            imm.save(os.path.join(empty_folder_path,str(str_empty_check[1:])+'.jpeg'))

        else:
        
            image = cv2.imread(os.path.join(out_path_images,str(empty_check)+'.jpeg'))
            #plt.imshow(image)
            imm = Image.fromarray(image)
            #imm.save("your_file.jpeg")
            imm.save(os.path.join(folder_path,str(str_empty_check[1:])+'.jpeg'))
            
end = time.time()

print(end - start)

1001.jpeg
1002.jpeg
new document
1003.jpeg
1004.jpeg
1005.jpeg
1006.jpeg
1007.jpeg
1008.jpeg
1009.jpeg
1010.jpeg
1011.jpeg
1012.jpeg
1013.jpeg
1014.jpeg
1015.jpeg
1016.jpeg
1017.jpeg
1018.jpeg
1019.jpeg
1020.jpeg
1021.jpeg
1022.jpeg
1023.jpeg
1024.jpeg
1025.jpeg
1026.jpeg
1027.jpeg
1028.jpeg
new document
1029.jpeg
1030.jpeg
1031.jpeg
1032.jpeg
1033.jpeg
1034.jpeg
1035.jpeg
1036.jpeg
1037.jpeg
1038.jpeg
1039.jpeg
1040.jpeg
1041.jpeg
1042.jpeg
1043.jpeg
1044.jpeg
1045.jpeg
1046.jpeg
1047.jpeg
1048.jpeg
1049.jpeg
1050.jpeg
1051.jpeg
1052.jpeg
1053.jpeg
1054.jpeg
1055.jpeg
1056.jpeg
1057.jpeg
1058.jpeg
new document
1059.jpeg
1060.jpeg
1061.jpeg
1062.jpeg
1063.jpeg
1064.jpeg
1065.jpeg
1066.jpeg
1067.jpeg
1068.jpeg
1069.jpeg
1070.jpeg
1071.jpeg
1072.jpeg
1073.jpeg
1074.jpeg
1075.jpeg
1076.jpeg
1077.jpeg
new document
1078.jpeg
1079.jpeg
1080.jpeg
1081.jpeg
1082.jpeg
1083.jpeg
1084.jpeg
1085.jpeg
1086.jpeg
1087.jpeg
1088.jpeg
1089.jpeg
1090.jpeg
1091.jpeg
1092.jpeg
1093.jpeg
1094.jpeg
new docu

In [16]:
import cv2
import os
import numpy as np
import pytesseract

In [22]:
im = cv2.imread('/home/vimalkumar/Downloads/new_project (copy)/output/batch_10-test/Files/014-024/020.jpeg',0)
_, blackAndWhite = cv2.threshold(im, 127, 255, cv2.THRESH_BINARY_INV)

nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
sizes = stats[1:, -1] #get CC_STAT_AREA component
img2 = np.zeros((labels.shape), np.uint8)

for i in range(0, nlabels - 1):
    if sizes[i] >= 50:   #filter small dotted regions
        img2[labels == i + 1] = 255

res = cv2.bitwise_not(img2)
#im = cv2.imread('/home/vimalkumar/Downloads/new_project/output_image/1002.jpeg',0)
image_data = pytesseract.image_to_string(res)
print(image_data)

out = " ".join(image_data.splitlines()).lower()
print(out)

out_replace = out.replace('  ','')
print(len(out_replace))



May 11, 2004

Scolaro,Shulman,Cohen,Fetter & Burstein

90 Presidential Plaza

Syracuse, NY 13202

Attn Alex

We have continued the title to the premises of Robert J Daino known as Lot 299
Lindenwood from April 14, 1997 to May 10, 2004 at 204 PM and find See Nos 1 thru
3 attached

We charge your account $ 175 00 for this service

Upon receipt of the abstract of title, we will enter the above Certification thereto

We appreciate your use of our services and trust that we may again serve you

Very truly yours,

MONROE TITLE INSURANCE CORPORATION

By

Authorized Signatory

No 300675

may 11, 2004  scolaro,shulman,cohen,fetter & burstein  90 presidential plaza  syracuse, ny 13202  attn alex  we have continued the title to the premises of robert j daino known as lot 299 lindenwood from april 14, 1997 to may 10, 2004 at 204 pm and find see nos 1 thru 3 attached  we charge your account $ 175 00 for this service  upon receipt of the abstract of title, we will enter the above certification ther

In [26]:
#im = cv2.imread(os.path.join(out_path_images,img),0)
#     _, blackAndWhite = cv2.threshold(im, 127, 255, cv2.THRESH_BINARY_INV)

#     nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
#     sizes = stats[1:, -1] #get CC_STAT_AREA component
#     img2 = np.zeros((labels.shape), np.uint8)

#     for i in range(0, nlabels - 1):
#         if sizes[i] >= 50:   #filter small dotted regions
#             img2[labels == i + 1] = 255

#     res = cv2.bitwise_not(img2)
#im = cv2.imread('/home/vimalkumar/Downloads/new_project/output_image/1002.jpeg',0)
image_data = pytesseract.image_to_string(im)
print(image_data)

out = " ".join(image_data.splitlines()).lower()
print(out)

out_replace = out.replace('  ','')
print(len(out_replace))



May 11, 2004

Scolaro,Shulman,Cohen,Fetter & Burstein

90 Presidential Plaza

Syracuse, NY 13202

Attn: Alex

We have continued the title to the premises of Robert J. Daino known as Lot 299
Lindenwood from April 14, 1997 to May 10, 2004 at 2:04 P.M. and find See Nos. 1 thru
3 attached.

We charge your account $ 175.00 for this service.

Upon receipt of the abstract of title, we will enter the above Certification thereto.

We appreciate your use of our services and trust that we may again serve you.

Very truly yours,

MONROE TITLE INSURANCE CORPORATION

By:

Authorized Signatory

No. 300675

may 11, 2004  scolaro,shulman,cohen,fetter & burstein  90 presidential plaza  syracuse, ny 13202  attn: alex  we have continued the title to the premises of robert j. daino known as lot 299 lindenwood from april 14, 1997 to may 10, 2004 at 2:04 p.m. and find see nos. 1 thru 3 attached.  we charge your account $ 175.00 for this service.  upon receipt of the abstract of title, we will enter the abov